In [1]:
from models import EfficientNet3D
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
batch_size = 8
inputs_shape = (batch_size, 45, 6, 6, 6)
labels_shape = (batch_size,)
inputs = torch.randn(inputs_shape).to(device)
labels = torch.randint(low=0, high=26, size=labels_shape).to(device)

cnn_flattened_size = inputs_shape[1] * inputs_shape[2] * inputs_shape[3] * inputs_shape[4]

print("inputs shape", inputs.shape)
print("labels shape", labels.shape)

inputs shape torch.Size([8, 45, 6, 6, 6])
labels shape torch.Size([8])


In [4]:
num_classes = 27
model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': num_classes}, 
                                 in_channels=1, hidden_size=128, task="classification")
print(sum([p.numel() for p in model.parameters()]))

4741096


In [6]:
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.train()

EfficientNet3D(
  (_conv_stem): Conv3dStaticSamePadding(
    1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1, 0, 1))
  )
  (_bn0): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock3D(
      (_depthwise_conv): Conv3dStaticSamePadding(
        32, 32, kernel_size=(3, 3, 3), stride=[2, 2, 2], groups=32, bias=False
        (static_padding): ZeroPad2d((0, 1, 0, 1, 0, 1))
      )
      (_bn1): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv3dStaticSamePadding(
        32, 8, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv3dStaticSamePadding(
        8, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv3dStaticSamePadding(
        32, 24, kernel_siz

In [7]:
previous_prediction_1 = torch.randn((batch_size, num_classes))
previous_prediction_2 = torch.randn((batch_size, num_classes))
# Concatenate the previous predictions together along dimension 2
previous_predictions = torch.cat((previous_prediction_1, previous_prediction_2), dim=1).cuda()
print("Previous prediction size outside", previous_predictions.shape)

Previous prediction size outside torch.Size([8, 54])


In [8]:
for epoch in range(1):
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs, previous_predictions)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    print('[%d] loss: %.3f' % (epoch + 1, loss.item()))

print('Finished Training')
print(outputs.shape)
print(outputs.cpu().detach().numpy())
print(outputs.cpu().detach().numpy().sum(axis=1))

[1] loss: 3.300
Finished Training
torch.Size([8, 27])
[[0.00985324 0.2189163  0.01097384 0.04642019 0.00879625 0.00845034
  0.0118514  0.00858997 0.01236258 0.2197035  0.00907216 0.00863427
  0.01577826 0.0306583  0.07196753 0.00896132 0.0155595  0.00664667
  0.14881474 0.00695251 0.01201527 0.03019348 0.02134095 0.02450305
  0.00885784 0.01063012 0.01349641]
 [0.10502825 0.00976233 0.16661155 0.04340981 0.00690355 0.0223591
  0.00930132 0.00674165 0.0097025  0.00719425 0.10208821 0.00677642
  0.00780932 0.03919661 0.00666501 0.0070331  0.00826254 0.02581845
  0.04563837 0.00745029 0.00668522 0.00702182 0.01407475 0.04211587
  0.15726662 0.12212791 0.00695522]
 [0.0699482  0.00900881 0.01020604 0.00576856 0.00800897 0.15810147
  0.01079067 0.00782115 0.01125611 0.01452929 0.00826018 0.00786148
  0.01032586 0.0078024  0.04689101 0.00976066 0.01116164 0.09367335
  0.01970655 0.09793743 0.12182268 0.00814618 0.16097473 0.05569447
  0.00806505 0.00723095 0.01924615]
 [0.01004563 0.03213231